In [1]:
import pandas as pd
import numpy as np

In [2]:
data22 = pd.read_csv("base_ano22.csv", sep=",")
data23 = pd.read_csv("base_ano23.csv", sep=",")
data24 = pd.read_csv("base_ano24.csv", sep=",")

/tmp/ipykernel_6944/1633524087.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data22 = pd.read_csv("base_ano22.csv", sep=",")
/tmp/ipykernel_6944/1633524087.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data23 = pd.read_csv("base_ano23.csv", sep=",")


# Data treatment

## 1) Preparing data before merging the dataframes from the years 2022, 2023 and 2024 together

No missing value in all dataframes

In [3]:
data24['tranche_age23'].isna().sum()

np.int64(0)

In [4]:
data22['nb_occ_id'] = data22.groupby('id_personne')['id_personne'].transform('count')
data23['nb_occ_id'] = data23.groupby('id_personne')['id_personne'].transform('count')
data24['nb_occ_id'] = data24.groupby('id_personne')['id_personne'].transform('count')

contain the rows from doublons22 where, for each group of 'id_personne', 'annee_soins', and 'Mois_soins', the 'sum_pres' value is the maximum within that group.

Doublons contains all the rows from the dataframes that have duplicate values in the 'id_personne', 'annee_soins', 'Mois_soins' and 'delta' columns.

## ?

In [5]:
doublons22 = data22[data22.duplicated(subset=['id_personne', 'annee_soins', 'Mois_soins', 'delta'], keep=False)]
doublons23 = data23[data23.duplicated(subset=['id_personne', 'annee_soins', 'Mois_soins', 'delta'], keep=False)]
doublons24 = data24[data24.duplicated(subset=['id_personne', 'annee_soins', 'Mois_soins', 'delta'], keep=False)]

Resultat contain the rows from doublons where, for each group of 'id_personne', 'annee_soins', and 'Mois_soins', the 'sum_pres' value is the maximum within that group.

In [6]:
results22 = doublons22.loc[doublons22.groupby(['id_personne', 'annee_soins', 'Mois_soins'])['sum_pres'].idxmax()]
results23 = doublons23.loc[doublons23.groupby(['id_personne', 'annee_soins', 'Mois_soins'])['sum_pres'].idxmax()]
results24 = doublons24.loc[doublons24.groupby(['id_personne', 'annee_soins', 'Mois_soins'])['sum_pres'].idxmax()]

We remove the rows in doublons that are not in results

In [7]:
index_to_drop22 = list(set(doublons22.index) - set(results22.index))
data22 = data22.drop(index_to_drop22)
index_to_drop23 = list(set(doublons23.index) - set(results23.index))
data23 = data23.drop(index_to_drop23)
index_to_drop24 = list(set(doublons24.index) - set(results24.index))
data24 = data24.drop(index_to_drop24)

We replace the values in the 'sum_pres' column of the data DataFrame with the maximum 'sum_pres' value within each 'id_personne' group.

## ?

In [8]:
data22['sum_pres'] = data22.groupby('id_personne')['sum_pres'].transform('max')
data23['sum_pres'] = data23.groupby('id_personne')['sum_pres'].transform('max')
data24['sum_pres'] = data24.groupby('id_personne')['sum_pres'].transform('max')

## ?

In [9]:
data22 = data22[~((data22['annee_soins'] == 0) & (data22['nb_occ_id'] != 1))]
data23 = data23[~((data23['annee_soins'] == 0) & (data23['nb_occ_id'] != 1))]
data24 = data24[~((data24['annee_soins'] == 0) & (data24['nb_occ_id'] != 1))]

Encoding the genre values

In [10]:
# data22['genre'] = data22['genre'].replace({'Homme': 0, 'Femme': 1, 'Enfant': 2})
# data23['genre'] = data23['genre'].replace({'Homme': 0, 'Femme': 1, 'Enfant': 2})
# data24['genre'] = data24['genre'].replace({'Homme': 0, 'Femme': 1, 'Enfant': 2})

Wa create 2 new variables : ra is the average refund and fra is the average amount left to pay to the client

In [11]:
data22['ra'] = np.where(data22['nb_acte'] != 0, data22['remb_mutuelle'] / data22['nb_acte'], 0)
data23['ra'] = np.where(data23['nb_acte'] != 0, data23['remb_mutuelle'] / data23['nb_acte'], 0)
data24['ra'] = np.where(data24['nb_acte'] != 0, data24['remb_mutuelle'] / data24['nb_acte'], 0)

data22['fra'] = np.where(data22['nb_acte'] != 0, data22['frais_reels'] / data22['nb_acte'], 0)
data23['fra'] = np.where(data23['nb_acte'] != 0, data23['frais_reels'] / data23['nb_acte'], 0)
data24['fra'] = np.where(data24['nb_acte'] != 0, data24['frais_reels'] / data24['nb_acte'], 0)

We create new columns to get the amount paid and perceived from the insurance for each month, we then do the same for the "fra" and "delta" variables

## ? mois 0

In [12]:
for mois in range(1, 13):
    mois_str = str(mois).zfill(2)
    nom_variable = 'ra_' + mois_str + '22' 
    data22[nom_variable] = data22.loc[data22['Mois_soins'] == float(mois), 'ra']

for mois in range(1, 13):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'ra_' + mois_str + '23' 
    data23[nom_variable] = data23.loc[data23['Mois_soins'] == float(mois), 'ra']

for mois in range(1, 9):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'ra_' + mois_str + '24' 
    data24[nom_variable] = data24.loc[data24['Mois_soins'] == float(mois), 'ra']

In [13]:
for mois in range(1, 13):
    mois_str = str(mois).zfill(2)
    nom_variable = 'fra_' + mois_str + '22' 
    data22[nom_variable] = data22.loc[data22['Mois_soins'] == float(mois), 'fra']

for mois in range(1, 13):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'fra_' + mois_str + '23' 
    data23[nom_variable] = data23.loc[data23['Mois_soins'] == float(mois), 'fra']

for mois in range(1, 9):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'fra_' + mois_str + '24' 
    data24[nom_variable] = data24.loc[data24['Mois_soins'] == float(mois), 'fra']

In [14]:
for mois in range(1, 13):
    mois_str = str(mois).zfill(2)  
    nom_variable = 'delta_' + mois_str + '22' 
    data22[nom_variable] = data22.loc[data22['Mois_soins'] == float(mois), 'delta']

for mois in range(1, 13):
    mois_str = str(mois).zfill(2)  
    nom_variable = 'delta_' + mois_str + '23' 
    data23[nom_variable] = data23.loc[data23['Mois_soins'] == float(mois), 'delta']

for mois in range(1, 9):
    mois_str = str(mois).zfill(2)  
    nom_variable = 'delta_' + mois_str  + '24' 
    data24[nom_variable] = data24.loc[data24['Mois_soins'] == float(mois), 'delta']

We now aggregate each dataframe on the 'id_personne' column to then merge them together

In [15]:
data22_agg= data22.groupby('id_personne').agg({
    'entite_eco': 'first', 
    'personne_morale': 'first',
    'annee_soins': 'first',
    'colloc': 'first',
    'adh_fac':'first', # why first?
    'type_cont':'first',
    'genre': 'first',
    'code_postal': 'first',
    'Age22': 'first',
    'type_benef': 'first',
    'tranche_age22': 'first',
    'Foyer22': 'first',
    'code_postal': 'first',
    'rg_benef': 'first',
    'PRES2201': 'max',
    'PRES2202': 'max',
    'PRES2203': 'max', # why max?
    'PRES2204': 'max',
    'PRES2205': 'max',
    'PRES2206': 'max',
    'PRES2207': 'max',
    'PRES2208': 'max',
    'PRES2209': 'max',
    'PRES2210': 'max',
    'PRES2211': 'max',
    'PRES2212': 'max',
    'nb_occ_id': 'first',
    'ra_0122': 'sum', 
    'ra_0222': 'sum',
    'ra_0322': 'sum',
    'ra_0422': 'sum', 
    'ra_0522': 'sum', 
    'ra_0622': 'sum',
    'ra_0722': 'sum', 
    'ra_0822': 'sum', 
    'ra_0922': 'sum',
    'ra_1022': 'sum', 
    'ra_1122': 'sum', 
    'ra_1222': 'sum',
    'fra_0122': 'sum', 
    'fra_0222': 'sum', 
    'fra_0322': 'sum',
    'fra_0422': 'sum', 
    'fra_0522': 'sum',
    'fra_0622': 'sum',
    'fra_0722': 'sum', 
    'fra_0822': 'sum', 
    'fra_0922': 'sum',
    'fra_1022': 'sum', 
    'fra_1122': 'sum', 
    'fra_1222': 'sum',
    'delta_0122': 'max', 
    'delta_0222': 'max',
    'delta_0322': 'max',
    'delta_0422': 'max', 
    'delta_0522': 'max', 
    'delta_0622': 'max',
    'delta_0722': 'max', 
    'delta_0822': 'max', 
    'delta_0922': 'max',
    'delta_1022': 'max', 
    'delta_1122': 'max', 
    'delta_1222': 'max',
})

In [16]:
data23_agg= data23.groupby('id_personne').agg({
    'entite_eco': 'first', 
    'personne_morale': 'first',
    'annee_soins': 'first',
    'colloc': 'first',
    'adh_fac':'first', # why first?
    'type_cont':'first',
    'genre': 'first',
    'code_postal': 'first',
    'Age23': 'first',
    'type_benef': 'first',
    'tranche_age23': 'first',
    'Foyer23': 'first',
    'code_postal': 'first',
    'rg_benef': 'first',
    'PRES2301': 'max',
    'PRES2302': 'max',
    'PRES2303': 'max', # why max?
    'PRES2304': 'max',
    'PRES2305': 'max',
    'PRES2306': 'max',
    'PRES2307': 'max',
    'PRES2308': 'max',
    'PRES2309': 'max',
    'PRES2310': 'max',
    'PRES2311': 'max',
    'PRES2312': 'max',
    'nb_occ_id': 'first',
    'ra_0123': 'sum', 
    'ra_0223': 'sum',
    'ra_0323': 'sum',
    'ra_0423': 'sum', 
    'ra_0523': 'sum', 
    'ra_0623': 'sum',
    'ra_0723': 'sum', 
    'ra_0823': 'sum', 
    'ra_0923': 'sum',
    'ra_1023': 'sum', 
    'ra_1123': 'sum', 
    'ra_1223': 'sum',
    'fra_0123': 'sum', 
    'fra_0223': 'sum', 
    'fra_0323': 'sum',
    'fra_0423': 'sum', 
    'fra_0523': 'sum',
    'fra_0623': 'sum',
    'fra_0723': 'sum', 
    'fra_0823': 'sum', 
    'fra_0923': 'sum',
    'fra_1023': 'sum', 
    'fra_1123': 'sum', 
    'fra_1223': 'sum',
    'delta_0123': 'max', 
    'delta_0223': 'max',
    'delta_0323': 'max',
    'delta_0423': 'max', 
    'delta_0523': 'max', 
    'delta_0623': 'max',
    'delta_0723': 'max', 
    'delta_0823': 'max', 
    'delta_0923': 'max',
    'delta_1023': 'max', 
    'delta_1123': 'max', 
    'delta_1223': 'max',
})

In [17]:
data24_agg= data24.groupby('id_personne').agg({
    'entite_eco': 'first', 
    'personne_morale': 'first',
    'annee_soins': 'first',
    'colloc': 'first',
    'adh_fac':'first', # why first?
    'type_cont':'first',
    'genre': 'first',
    'code_postal': 'first',
    'Age24': 'first',
    'type_benef': 'first',
    'tranche_age23': 'first',
    'Foyer24': 'first',
    'code_postal': 'first',
    'rg_benef': 'first',
    'PRES2401': 'max',
    'PRES2402': 'max',
    'PRES2403': 'max', # why max?
    'PRES2404': 'max',
    'PRES2405': 'max',
    'PRES2406': 'max',
    'PRES2407': 'max',
    'PRES2408': 'max',
    'PRES2409': 'max',
    'PRES2410': 'max',
    'PRES2411': 'max',
    'PRES2412': 'max',
    'nb_occ_id': 'first',
    'ra_0124': 'sum', 
    'ra_0224': 'sum',
    'ra_0324': 'sum',
    'ra_0424': 'sum', 
    'ra_0524': 'sum', 
    'ra_0624': 'sum',
    'ra_0724': 'sum', 
    'ra_0824': 'sum',  
    'fra_0224': 'sum', 
    'fra_0324': 'sum',
    'fra_0424': 'sum', 
    'fra_0524': 'sum',
    'fra_0624': 'sum',
    'fra_0724': 'sum', 
    'fra_0824': 'sum', 
    'delta_0124': 'max', 
    'delta_0224': 'max',
    'delta_0324': 'max',
    'delta_0424': 'max', 
    'delta_0524': 'max', 
    'delta_0624': 'max',
    'delta_0724': 'max', 
    'delta_0824': 'max', 
})

Before merging we replace all the Nans in the ra, fra and delta columns by zero to make the difference between people who did not perceive any money from the insurance and those who where not attached to it after the merge

In [18]:
data22_agg.fillna(0, inplace=True)
data23_agg.fillna(0, inplace=True)
data24_agg.fillna(0, inplace=True)

## 2) Merging the dataframes 

We new proceed to merge the three dataframes while also removing the duplicate columns

In [19]:
data22_23_merged = pd.merge(data22_agg, data23_agg, on='id_personne', how='outer', suffixes=('', '_y'))

We also need to make sure that values that only appear in the duplicate columns of data23_agg are updated in the merged dataframe before removing these columns.

In [20]:
for col in data22_agg.columns:
    if col != 'id_personne' and col + '_y' in data22_23_merged.columns:
        data22_23_merged[col] = data22_23_merged[col].combine_first(data22_23_merged[col + '_y'])

In [21]:
data22_23_merged.drop(data22_23_merged.filter(regex='_y$').columns, axis=1, inplace=True)

In [22]:
full_data = pd.merge(data22_23_merged, data24_agg, on='id_personne', how='outer', suffixes=('', '_y'))

In [23]:
for col in data22_23_merged.columns:
    if col != 'id_personne' and col + '_y' in full_data.columns:
        full_data[col] = full_data[col].combine_first(full_data[col + '_y'])

In [24]:
full_data.drop(full_data.filter(regex='_y$').columns, axis=1, inplace=True)

We now have the final dataframe we will use for clustering 

In [25]:
full_data.head()

,entite_eco,personne_morale,annee_soins,colloc,adh_fac,type_cont,genre,code_postal,Age22,type_benef,...,fra_0724,fra_0824,delta_0124,delta_0224,delta_0324,delta_0424,delta_0524,delta_0624,delta_0724,delta_0824
id_personne,,,,,,,,,,,,,,,,,,,,,
0000031a283216ec643bea5b46511325bb088033d75a379188a37162cf4ebb10,EE7,PM7,2022.0,non,non,0.0,Femme,13540.0,45.0,Salarié,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00002650c3250e7fe5b197f45a27ab2e4fc38c1cdee65ce8e08a43186b8ac4b8,EE2703,PM3563,2022.0,non,non,0.0,Enfant,41000.0,10.0,Enfant mineur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000329a0a7e29081ee5dafa006d627f7eb4db82c046ad2f36f609ccd3101a63,EE1979,PM2,2022.0,0,0,0.0,Enfant,99999.0,3.0,Enfant mineur,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00003cd2efe009e8bfea54a7de13f162e5359495da8cf9f04258104c845ffdfe,EE2948,PM3908,2022.0,non,non,0.0,Enfant,75018.0,26.0,Enfant majeur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000458142f363a32276d020258f637fa4b173c8471e6e73049eadea49753db1,EE2149,PM2680,2022.0,non,non,0.0,Homme,33400.0,32.0,Salarié,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3) Encoding the categorical variables in the columns for clustering

We will first encode the 'type_benef' column. We will use one-hot encoding as this column takes only 5 different values.

In [26]:
set(full_data['type_benef'].values)

{'Autre', 'Conjoint', 'Enfant majeur', 'Enfant mineur', 'Salarié'}

In [27]:
full_data = pd.get_dummies(full_data, columns=['type_benef'], prefix='category')

We will now use the same encoding for the "genre" and "Foyer22" columns as they have only four and ten distinct values

In [29]:
set(full_data['genre'].values)

{'Enfant', 'Femme', 'Homme', 'Pas de donnees'}

In [30]:
full_data = pd.get_dummies(full_data, columns=['genre'], prefix='category')

In [32]:
set(full_data['Foyer22'].values)

{'ND',
 'Salarié et 1 enfant',
 'Salarié et 2 enfants',
 'Salarié et 3+ enfants',
 'Salarié et conjoint et 1 enfant',
 'Salarié et conjoint et 2 enfants',
 'Salarié et conjoint et 3+ enfants',
 'Salarié et conjoint sans enfant',
 'Salarié seul',
 nan}

In [33]:
full_data = pd.get_dummies(full_data, columns=['Foyer22'], prefix='category')

We will now encode the 'tranche_age22' column. This time we will use ordinal encoding to keep a relation of order between the distinct values the vartiable takes

In [38]:
set(full_data['tranche_age22'].values)

{'[0; 18[',
 '[18; 24[',
 '[20.0; 25.0[',
 '[25.0; 30.0[',
 '[30.0; 35.0[',
 '[35.0; 40.0[',
 '[40.0; 45.0[',
 '[45.0; 50.0[',
 '[50.0; 55.0[',
 '[55.0; 60.0[',
 '[60.0; 65.0[',
 '[65 et +[',
 nan}

In [39]:
full_data['tranche_age22'] = full_data['tranche_age22'].replace({'[0; 18[': 1, '[18; 24[': 2, '[20.0; 25.0[': 3, '[25.0; 30.0[': 4, '[30.0; 35.0[': 5, '[35.0; 40.0[': 6,
                                                                '[40.0; 45.0[': 7, '[45.0; 50.0[': 8, '[50.0; 55.0[': 9, '[55.0; 60.0[': 10, '[60.0; 65.0[': 11, '[65 et +[': 12})

/tmp/ipykernel_6944/3330550080.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_data['tranche_age22'] = full_data['tranche_age22'].replace({'[0; 18[': 1, '[18; 24[': 2, '[20.0; 25.0[': 3, '[25.0; 30.0[': 4, '[30.0; 35.0[': 5, '[35.0; 40.0[': 6,


## Que faire des nan (211 000 nans)